In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
data = pd.read_csv('가사전처리.csv', low_memory=False)

keep_col = ["제목","가수","장르","발매일","앨범명","가사"]
new_data = data[keep_col].replace('\\n',' ', regex=True)
new_data.to_csv("newFile.csv", index=False,encoding = 'utf-8-sig')
data.head(2)

,제목,가수,장르,발매일,앨범명,가사
0,이산혜연선사발원문,영인스님,종교음악,20101207,영인스님 염불 따라하기 1 (예불/천수경),시방 삼세 부처님과 팔만사천 큰 법보와\n\n \n\n보살 성문 스님네께 지성 귀...
1,사노라면,쟈니리,성인가요,20090501,걱정마,\n흐린 날도 날이 새면 행복하지 않던가\n\n새파랗게 젊다는게 한 밑천인데\n쩨쩨...


In [3]:
data = data.head(20000)

In [4]:
if data['가사'].isnull().sum() :
    data['가사'] = data['가사'].fillna('')
else :
    print("결측값 없음")

결측값 없음


In [5]:
data.index 

RangeIndex(start=0, stop=20000, step=1)

In [6]:
# ngram_range = (1,2) 단어의 묶음을 1개부터 2개까지 설정
tfidf = TfidfVectorizer(stop_words='english',analyzer = 'word',  min_df=2, ngram_range = (1, 2), sublinear_tf=True)

# 가사에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['가사'])
print(tfidf_matrix.shape)

tfidf.fit(data['가사']) # 벡터라이저가 단어들을 학습합니다.
sorted(tfidf.vocabulary_.items(),reverse=True) # 단어사전을 정렬합니다. 
tfidf.vocabulary_ # 벡터라이저가 학습한 단어사전을 출력합니다. 


(20000, 210295)


{'시방': 127938,
 '성문': 120581,
 '지성': 184927,
 '자비하신': 173704,
 '주옵소서': 181856,
 '참된': 188154,
 '성품': 120624,
 '뛰어들어': 76729,
 '나고': 40582,
 '죽는': 182201,
 '물결따라': 93486,
 '빛과': 108478,
 '소리': 121779,
 '물이': 93820,
 '들고': 72272,
 '온갖': 154130,
 '번뇌': 101063,
 '보고': 102410,
 '듣고': 72130,
 '한량없는': 200372,
 '죄를': 180890,
 '지어': 184944,
 '잘못된': 174688,
 '갈팡질팡': 16606,
 '나와': 44144,
 '남을': 47206,
 '집착하고': 186711,
 '길만': 36803,
 '찾아다녀': 188850,
 '여러': 149633,
 '생에': 119046,
 '지은': 185396,
 '크고': 192934,
 '작은': 174179,
 '많은': 81078,
 '허물': 205251,
 '부처님이': 106130,
 '이끄시고': 162241,
 '고통': 24318,
 '바다': 96140,
 '열반': 150717,
 '언덕': 144682,
 '세상에': 120963,
 '명과': 87391,
 '길이길이': 37184,
 '오는': 152429,
 '세상': 120690,
 '불법': 106683,
 '지혜': 186056,
 '무럭무럭': 92398,
 '자라나서': 173469,
 '좋은': 180614,
 '밝은': 99211,
 '스승': 126083,
 '만나오며': 80048,
 '바른': 97724,
 '굳게': 26107,
 '세워': 121426,
 '귀와': 26468,
 '눈이': 60187,
 '말과': 81654,
 '뜻이': 77058,
 '세상일에': 121387,
 '청정': 190318,
 '닦고': 64085,
 '닦아': 64098,
 '엄한

In [7]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) 

In [42]:
indices = pd.Series(data.index,index= [ data['제목'], data['가수']]  ).drop_duplicates() 
   
print(indices.index)    

print(indices.head())

MultiIndex([(             '이산혜연선사발원문',   '영인스님'),
            (                  '사노라면',    '쟈니리'),
            (          '우리들이 함께 있는 밤',    '박기영'),
            (              '밤의 고독속에서',    '김현식'),
            (               '아이돌이 좋아',    '임정우'),
            (              '별것도 아닌 일',    '이진아'),
            ('Baby You Are Beautiful',    '성시경'),
            (                 '나는 너를',    '한승기'),
            (         'Psycho heaven',    '자우림'),
            (                '떠나는 저녁',    '이승철'),
            ...
            (                '바람이 분다',    '폰부스'),
            (               '번지없는 주막',    '백년설'),
            (                     '왜',    '장혜진'),
            (               '카스바의 여인',    '함중아'),
            (                  '나야나야',    '정수성'),
            (              '이 마음 다하여',   '세실리아'),
            (                 '살다가보면',     '정인'),
            (                    '자위',  '푸른 새벽'),
            (                  '봄의기적',    '이지형'),
            (                   '경

In [43]:
target_singer = '길구봉구'
target_singer = target_singer.strip()
target_title = '있어줄래'
target_title = target_title.strip()
target_genre = '발라드'
target_genre = target_genre.strip()
idx = indices[(target_title, target_singer)][0]
print(idx)

17710


In [46]:
def get_recommendations(title, singer, genre, cosine_sim=cosine_sim):
    # 선택한 음악의 가수, 제목으로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 음악를 가지고 연산할 수 있습니다.
    #idx = indices[title]
    idx = indices[(title, singer)][0]
    
    # 모든 음악에 대해서 해당 음악과의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))
 
    # 유사도에 따라 음악들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 100개의 음악을 받아옵니다.
    sim_scores = sim_scores[1:101]

    #코사인값은 두 벡터의 방향이 완전히 같을 경우 1, 90°의 각을 이룰 경우 0, 180°로 완전히 반대 방향인 경우 -1의 값을 갖음
    #print(sim_scores)
    
    # 가장 유사한 100개의 음악의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 곡 출력
    num = 0
    for i in range(len(movie_indices)) :
        if num == 11 :
            break
        if data['장르'].iloc[movie_indices[i]] !=  genre :
            continue
        num += 1
        a = data['가수'].iloc[movie_indices[i]] 
        b = data['제목'].iloc[movie_indices[i]]
        c = data['장르'].iloc[movie_indices[i]]
        print('['+a+'] '+b+' ('+c+')') 

In [47]:
get_recommendations(target_title, target_singer, target_genre)

[홍성민] 기억날 그날이 와도 (발라드)
[한경일] 반지 (발라드)
[유미] 소녀의 고백 (발라드)
[포피] 꿈꿔왔었던 (발라드)
[박현서] 밤 중 (발라드)
[포지션] 하루 (발라드)
[김준식] 일상 (Daily) (발라드)
[바이브] Because I Love You (발라드)
[별] Stay By My Side (발라드)
[손효규] 살짝 (발라드)
[해피체어] 여기 추억이 있다 (발라드)
